In [32]:
#!/usr/bin/env python3
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from matplotlib.backends.backend_pdf import PdfPages
pdf_pages = None

In [22]:


def get_sorted_frames(frames_dir):
    # Get full paths for PNG files, sort alphabetically (or numerically if framed well-named)
    frame_files = sorted([os.path.join(frames_dir, f)
                          for f in os.listdir(frames_dir)
                          if f.lower().endswith(".png")])
    return frame_files

In [23]:

def sample_frames(frame_files, sample_count=5):
    num_frames = len(frame_files)
    if num_frames == 0:
        return []
    
    # If there are fewer frames than sample_count, adjust sample_count
    sample_count = min(sample_count, num_frames)
    # Calculate indices to sample (roughly equally spaced)
    indices = [int(round(i * (num_frames - 1) / (sample_count - 1))) 
               for i in range(sample_count)]
    sampled = [frame_files[i] for i in indices]
    return sampled

In [24]:
# def add_directory_text(image, text):
#     # Overlay the camera directory name on the top-left corner of the image
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(image, text, (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
#     return image

In [29]:
def display_frames_side_by_side(frames, directory_name):
    imgs = []
    for frame_path in frames:
        img = cv2.imread(frame_path)
        if img is None:
            print(f"Error reading {frame_path}")
            continue
        # Optionally resize the frame for consistency
        img = cv2.resize(img, (320, 240))
        # Convert image color space from BGR to RGB for matplotlib compatibility
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Add overlay text using OpenCV (if needed)
        # img = add_directory_text(img, directory_name)
        imgs.append(img)
    
    if not imgs:
        return
    # Stack images horizontally
    combined = np.hstack(imgs)
    
    # Display the image inline using matplotlib
    plt.figure(figsize=(15, 5))
    plt.imshow(combined)
    plt.title(f"Frames from: {directory_name}")
    plt.axis('off')
    fig = plt.gcf()
    if pdf_pages is not None:
        pdf_pages.savefig(fig)
    plt.close(fig)
    plt.show()
    
    # NEW: Save current figure to PDF if pdf_pages is available
    

In [36]:
def process_camera_directory(camera_path):
 # 1. Check if camera folder is "flir" (case-insensitive). Delete if it is.
    camera_name = os.path.basename(camera_path)
    if camera_name.lower() == 'flir':
        print(f"Deleting {camera_path} because the camera folder is named 'flir'.")
        shutil.rmtree(camera_path)
        return

    # 2. Check if frames directory exists.
    frames_dir = os.path.join(camera_path, "frames")
    if not os.path.isdir(frames_dir):
        print(f"No frames directory found in {camera_path}. Deleting the camera folder.")
        shutil.rmtree(camera_path)
        return

    # 3. Get sorted PNG frames from the frames directory.
    frame_files = get_sorted_frames(frames_dir)
    if (not frame_files) or (len(frame_files)<2):
        print(f"No PNG frames found in {frames_dir}. Deleting the camera folder.")
        shutil.rmtree(camera_path)
        return

    # 4. Sample frames (default: 3 frames)
    sampled_frames = sample_frames(frame_files, sample_count=3)
    
    # NEW: Plot rotated image based on subject id (parent folder of camera_path)
    subject_id = os.path.basename(os.path.dirname(camera_path))
    rotated_img_file = os.path.join('/home/caio.dasilva/datasets/brc2_rotate', subject_id + "_set1_wb0_1_0.rs-image-5.png")
    if os.path.isfile(rotated_img_file):
        img_rot = cv2.imread(rotated_img_file)
        if img_rot is not None:
            img_rot = cv2.cvtColor(img_rot, cv2.COLOR_BGR2RGB)
            # NEW: Resize rotated image to match the size of other frames
            img_rot = cv2.resize(img_rot, (320, 240))
            plt.figure()
            plt.imshow(img_rot)
            plt.title(f"Rotated image for {subject_id}")
            plt.axis('off')
            fig = plt.gcf()
            if pdf_pages is not None:
                pdf_pages.savefig(fig)
            plt.close(fig)
            plt.show()
            # NEW: Save current rotated image figure to PDF if pdf_pages is available
            
        else:
            print(f"Error reading rotated image {rotated_img_file}")
    else:
        print(f"No rotated image found for subject {subject_id} at {rotated_img_file}")
    
    # 5. Display the sampled frames side by side with the camera name overlayed.
    display_frames_side_by_side(sampled_frames, camera_path)

In [27]:
def main(root_dir):
    if not os.path.isdir(root_dir):
        print(f"{root_dir} is not a directory.")
        return

    # Iterate over subject directories
    for subject in os.listdir(root_dir):
        subject_path = os.path.join(root_dir, subject)
        if not os.path.isdir(subject_path):
            continue

        # Iterate over camera directories inside each subject directory
        for camera in os.listdir(subject_path):
            camera_path = os.path.join(subject_path, camera)
            if not os.path.isdir(camera_path):
                continue
            process_camera_directory(camera_path)

#### if __name__ == "__main__":
    root_directory = input("Enter the root directory path: ").strip()
    with PdfPages("output.pdf") as temp_pdf:
        pdf_pages = temp_pdf
        main(root_directory)